In [334]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
#import seaborn
#seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
import matplotlib.ticker
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 4000) 

## S4 H2 adsorption isotherm, lumped injections

### Reading in the data and preparing DataFrames

In [335]:
#reading vaclogger measurement file
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()

#adding an elapsed time column
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

print(vaclog)


                                     Live comments                Time  \
0                                              NaN 2023-02-23 12:38:12   
1     inj volume prepared, baseline mass scan done 2023-02-23 12:38:19   
2                                              NaN 2023-02-23 12:38:26   
3                                              NaN 2023-02-23 12:38:33   
4                                              NaN 2023-02-23 12:38:40   
5                                              NaN 2023-02-23 12:38:46   
6                                              NaN 2023-02-23 12:38:53   
7                                              NaN 2023-02-23 12:39:00   
8                                              NaN 2023-02-23 12:39:07   
9                                              NaN 2023-02-23 12:39:14   
10                                             NaN 2023-02-23 12:39:20   
11                                             NaN 2023-02-23 12:39:27   
12                                    

### Data processing

In [336]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'inj volume prepared, baseline mass scan done' 'RGA filament off'
 'turbo off' 'bp check']


#### CernOx temperature R-T conversion

In [337]:
#Temperature curve for CERNOX 
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
# specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


## Ignore end of measurement

In [338]:
#slicing the dataset
ramp = vaclog.iloc[list((vaclog["Elapsed time"]/3600).between(0,2))]
print(ramp)

                                     Live comments                Time  \
0                                              NaN 2023-02-23 12:38:12   
1     inj volume prepared, baseline mass scan done 2023-02-23 12:38:19   
2                                              NaN 2023-02-23 12:38:26   
3                                              NaN 2023-02-23 12:38:33   
4                                              NaN 2023-02-23 12:38:40   
5                                              NaN 2023-02-23 12:38:46   
6                                              NaN 2023-02-23 12:38:53   
7                                              NaN 2023-02-23 12:39:00   
8                                              NaN 2023-02-23 12:39:07   
9                                              NaN 2023-02-23 12:39:14   
10                                             NaN 2023-02-23 12:39:20   
11                                             NaN 2023-02-23 12:39:27   
12                                    

## Subtracting baseline

In [339]:
#TODO - calculating coverages for plotting pressure v coverage
#in the first assumption coverage is assumed to be equivalent to the number of molecules injected
#coverage at any given time point in [M/cm2] -> M/s * total injection time 
#First - Determine the inj start point, set elapsed time here to 0. 
inj = vaclog[(vaclog["Barion_2"]).between(1e-8,3e-6)]
inj.drop(columns="Elapsed time", inplace=True)
print(inj)
timestamps2 = pd.to_datetime(inj["Time"],format="%d/%m/%Y %H:%M:%S")
runtime2 = (timestamps2-timestamps2[91]).dt.total_seconds()    #hardcoded value
inj.insert(1,"Elapsed time",runtime2)



C:\Users\etiirine\AppData\Local\Temp\ipykernel_1940\440027113.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inj.drop(columns="Elapsed time", inplace=True)


     Live comments                Time  injection 100mbar      Barion_2  \
91             NaN 2023-02-23 12:48:42             88.990  1.140000e-08   
92             NaN 2023-02-23 12:48:48             88.986  1.800000e-07   
93             NaN 2023-02-23 12:48:55             88.985  1.300000e-07   
94             NaN 2023-02-23 12:49:02             88.985  1.890000e-07   
95             NaN 2023-02-23 12:49:09             88.985  2.670000e-07   
96             NaN 2023-02-23 12:49:16             88.981  3.110000e-07   
97             NaN 2023-02-23 12:49:22             88.981  5.170000e-07   
98             NaN 2023-02-23 12:49:29             88.973  5.970000e-07   
99             NaN 2023-02-23 12:49:36             88.965  8.880000e-07   
100            NaN 2023-02-23 12:49:43             88.949  2.050000e-06   
101            NaN 2023-02-23 12:49:50             88.929  2.090000e-06   
102            NaN 2023-02-23 12:49:56             88.913  2.240000e-06   
103            NaN 2023-0

In [340]:
#adding H2 equiv pressure readings
inj.insert(3, "corr BA2", inj["Barion_2"]/0.46)
inj.insert(4, "corr BA1", inj["Barion_1"]/0.46)


### Thermal transpiration
Applied for the gauge readings using the formula below:
    $$
    \frac{p_2}{p_1}=\sqrt \frac{T_2}{T_1}
    $$
Here $p_2$ and $T_2$ are the pressure and temperature in the cold part and $p_1$, $T_1$ are the pressure, temperature the gauge is exposed to.

In [341]:
#implementing the formula
T2 = 4.2
T1 = 293
p_coef = np.sqrt(T2/T1)
inj.insert(3, "H2 4K BA2", (inj["corr BA2"]*p_coef))
inj.insert(4, "H2 4K BA1", (inj["corr BA1"]*p_coef))


In [342]:
print(p_coef)
print(inj)

0.11972665112564158
     Live comments  Elapsed time                Time     H2 4K BA2  \
91             NaN           0.0 2023-02-23 12:48:42  2.967139e-09   
92             NaN           6.0 2023-02-23 12:48:48  4.684956e-08   
93             NaN          13.0 2023-02-23 12:48:55  3.383579e-08   
94             NaN          20.0 2023-02-23 12:49:02  4.919204e-08   
95             NaN          27.0 2023-02-23 12:49:09  6.949351e-08   
96             NaN          34.0 2023-02-23 12:49:16  8.094563e-08   
97             NaN          40.0 2023-02-23 12:49:22  1.345623e-07   
98             NaN          47.0 2023-02-23 12:49:29  1.553844e-07   
99             NaN          54.0 2023-02-23 12:49:36  2.311245e-07   
100            NaN          61.0 2023-02-23 12:49:43  5.335644e-07   
101            NaN          68.0 2023-02-23 12:49:50  5.439754e-07   
102            NaN          74.0 2023-02-23 12:49:56  5.830167e-07   
103            NaN          81.0 2023-02-23 12:50:03  5.986333e-07   


In [343]:
#define baseline assuming that pressure during injection >> base pressure
#find the first derivative (rate of change)
derivative1 = np.diff(inj["Barion_1"])
print(derivative1)
derivative2 = np.diff(inj["Barion_2"])
#find the baseline indices below the threshold value (large deltas occur during injection)
threshold = 5e-8
baseline_ind1 = np.where(derivative1 > threshold)[0]
baseline_ind2 = np.where(derivative2 > threshold)[0]
#Split vaclog dataframe into two parts based on the identified baseline indices
base1 = inj.iloc[baseline_ind1]
base2 = inj.iloc[baseline_ind2]
print(base1)
#take the average of the baseline
avg1 = np.mean(base1["corr BA1"])
avg2 = np.mean(base2["corr BA2"])
print(avg1)
print(avg2)


[ 1.661e-07 -4.700e-08  6.400e-08 ...  0.000e+00  0.000e+00  0.000e+00]
    Live comments  Elapsed time                Time     H2 4K BA2  \
91            NaN           0.0 2023-02-23 12:48:42  2.967139e-09   
93            NaN          13.0 2023-02-23 12:48:55  3.383579e-08   
94            NaN          20.0 2023-02-23 12:49:02  4.919204e-08   
95            NaN          27.0 2023-02-23 12:49:09  6.949351e-08   
96            NaN          34.0 2023-02-23 12:49:16  8.094563e-08   
97            NaN          40.0 2023-02-23 12:49:22  1.345623e-07   
98            NaN          47.0 2023-02-23 12:49:29  1.553844e-07   
99            NaN          54.0 2023-02-23 12:49:36  2.311245e-07   
101           NaN          68.0 2023-02-23 12:49:50  5.439754e-07   
102           NaN          74.0 2023-02-23 12:49:56  5.830167e-07   

        H2 4K BA1      corr BA2      corr BA1  injection 100mbar  \
91   4.398653e-09  2.478261e-08  3.673913e-08             88.990   
93   3.539744e-08  2.826087e-07 

In [344]:
#subtract the baseline from the graph to get the actual pressure evolution on the sample

#Plotting the h2 equiv base pressure subtracted data
fig, ax = plt.subplots()
#subtract the baseline from the graph to get the actual pressure evolution on the sample

ax.plot(inj["Time"],inj['H2 4K BA1'].sub(avg1),marker=".", markersize=5,label='BA1')
ax.plot(inj["Time"],inj['H2 4K BA2'].sub(avg2),marker=".", markersize=5,label='BA2')
plt.legend(loc="upper left")

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.legend(loc="upper right")
plt.title("Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ")


Text(0.5, 1.0, 'Pressure evolution on the sample, p increase from inj flux subtracted. \n Pressures H2 equivalent, thermal transp. corrected ')

### Injection pressure linearity

In [345]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = inj["injection 100mbar"].values
x = (inj["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = -0.57*x+85.05


### Formula for sum of the differential injection flux
$$
N=\frac{\Sigma dp\cdot V_{inj}}{k_B\cdot T}=\frac{\Sigma (p_{inj,t(x)}-p_{inj,t(x-1)})\cdot V_{inj}}{k_B \cdot T}   \space \left[{M}\right]
$$

In [346]:
#constants in SI units
T = 293
k_B = 1.38e-23
V_inj = 6.515e-5
S_sample= 276
#implement the formula above
dp = abs(np.diff(inj["injection 100mbar"]*100))
print(np.mean(dp))
dp = np.insert(dp,0,0)
N_x = (dp.cumsum())*V_inj/(k_B*T)
print(N_x)
N_cov = (N_x/S_sample)
print(N_cov)


0.3974683544304519
[0.00000000e+00 6.44507098e+15 8.05633872e+15 ... 1.06198657e+19
 1.06198657e+19 1.06246995e+19]
[0.00000000e+00 2.33517064e+13 2.91896331e+13 ... 3.84777743e+16
 3.84777743e+16 3.84952881e+16]


In [347]:
inj.insert(2, "Coverage",N_cov)


In [348]:
inj.head()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
91,NaN,0.0,0.000000e+00,2023-02-23 12:48:42,2.967139e-09,4.398653e-09,2.478261e-08,3.673913e-08,88.990,1.140000e-08,1.690000e-08,5.343000e-08,3.718000e-07,1.1,451.0,-5.275,4273.384,NaN,NaN
92,NaN,6.0,2.335171e+13,2023-02-23 12:48:48,4.684956e-08,4.763039e-08,3.913043e-07,3.978261e-07,88.986,1.800000e-07,1.830000e-07,1.026000e-07,3.718000e-07,1.1,451.0,-5.387,4273.384,NaN,NaN
93,NaN,13.0,2.918963e+13,2023-02-23 12:48:55,3.383579e-08,3.539744e-08,2.826087e-07,2.956522e-07,88.985,1.300000e-07,1.360000e-07,1.378000e-07,3.714000e-07,1.1,451.0,-5.275,4273.384,NaN,NaN
94,NaN,20.0,2.918963e+13,2023-02-23 12:49:02,4.919204e-08,5.205507e-08,4.108696e-07,4.347826e-07,88.985,1.890000e-07,2.000000e-07,1.971000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
95,NaN,27.0,2.918963e+13,2023-02-23 12:49:09,6.949351e-08,7.183599e-08,5.804348e-07,6.000000e-07,88.985,2.670000e-07,2.760000e-07,2.297000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN


In [349]:
## dropping coverage values close to 0
inj_new = inj.loc[inj['Coverage'] > 1e+13]
inj_new.head()
#inj_new["H2 4K BA2"].sub(avg2).describe()

,Live comments,Elapsed time,Coverage,Time,H2 4K BA2,H2 4K BA1,corr BA2,corr BA1,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
92,NaN,6.0,2.335171e+13,2023-02-23 12:48:48,4.684956e-08,4.763039e-08,3.913043e-07,3.978261e-07,88.986,1.800000e-07,1.830000e-07,1.026000e-07,3.718000e-07,1.1,451.0,-5.387,4273.384,NaN,NaN
93,NaN,13.0,2.918963e+13,2023-02-23 12:48:55,3.383579e-08,3.539744e-08,2.826087e-07,2.956522e-07,88.985,1.300000e-07,1.360000e-07,1.378000e-07,3.714000e-07,1.1,451.0,-5.275,4273.384,NaN,NaN
94,NaN,20.0,2.918963e+13,2023-02-23 12:49:02,4.919204e-08,5.205507e-08,4.108696e-07,4.347826e-07,88.985,1.890000e-07,2.000000e-07,1.971000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
95,NaN,27.0,2.918963e+13,2023-02-23 12:49:09,6.949351e-08,7.183599e-08,5.804348e-07,6.000000e-07,88.985,2.670000e-07,2.760000e-07,2.297000e-07,3.718000e-07,1.1,451.0,-5.275,4273.496,NaN,NaN
96,NaN,34.0,5.254134e+13,2023-02-23 12:49:16,8.094563e-08,8.927444e-08,6.760870e-07,7.456522e-07,88.981,3.110000e-07,3.430000e-07,3.930000e-07,3.718000e-07,1.1,450.0,-5.050,4273.608,NaN,NaN


# Plotting

In [350]:
#plotting parameters
font = {'family': 'arial',
        'color':  'green',
        'weight': 'normal',
        'size': 10,
        }

plt.rcParams["figure.figsize"] = [8, 5]
plt.rcParams["figure.autolayout"] = True
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "black"
plt.rcParams["grid.linewidth"] = 0.35
plt.rcParams["xtick.direction"] = "out"
plt.rcParams["xtick.major.width"] = 1
plt.rcParams["xtick.major.size"] = 4
plt.rcParams["ytick.direction"] = "out"
plt.rcParams["ytick.major.width"] = 1
plt.rcParams["ytick.major.size"] = 4
plt.rcParams["xtick.major.bottom"] = True
plt.rcParams["ytick.major.left"] = True
plt.rcParams["axes.grid.axis"] = "both"
plt.rcParams["axes.grid.which"] = "both"



In [351]:
#Plotting the general pressure evolution and temperature for overview
fig, ax = plt.subplots()

l5 = ax.plot(vaclog["Time"], polyfit(A,vaclog["T-CERNOX"]),marker=".", color="cyan", markersize=5,label='Temperature')
ax.set_ylabel('Temperature (K)')
ax.set_xlabel('Timestamp')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

#Duplicate x axis for pressure values
ax2=ax.twinx()
l1 = ax2.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
l2 = ax2.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
l3 = ax2.plot(vaclog["Time"],vaclog['DUAL experiment'],marker=".", markersize=4,label='Dual gauge')
l4 = ax2.plot(vaclog["Time"],vaclog['injection 100mbar'],marker=".", markersize=4,label='100mbar F-R')
ax2.set_ylabel('Pressure (mbar)')
ax2.legend(handles = l1+l2+l3+l4+l5, loc="upper right")
ax2.set_yscale('log')

plt.title("Pressure evolution + temperatures")
plt.tight_layout()  # Ensure all elements are properly placed within the figure
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm_overview.png', dpi=300)
plt.show()


In [352]:
#Plotting BA2, BA1 pressure
fig, ax = plt.subplots()

ax.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
ax.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
ax.plot(vaclog["Time"],vaclog["DUAL experiment"],marker=".", markersize=4,label='DUAL experiment')
ax.plot(vaclog["Time"],vaclog['injection 100mbar'],marker=".", markersize=4,label='100mbar F-R')

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.legend()
ax.set_title("S4 CU ESD sample out - Total pressure evolution")

#ax.grid(which='major', axis='y')
#ax.get_yaxis().get_major_formatter().labelOnlyBase = False
#ax.tick_params(axis="y", which='major')
ax.set_yscale('log')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.tight_layout()  # Ensure all elements are properly placed within the figure
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm_Pressures.png',dpi=300)
plt.show()

In [353]:
#plotting the sliced data
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots()

ax.plot(ramp["Time"],ramp['Barion_1'],marker=".", markersize=4,label='BA1')
ax.plot(ramp["Time"],ramp['Barion_2'],marker=".", markersize=4,label='BA2')
ax.plot(ramp["Time"],ramp['DUAL experiment'],marker=".", markersize=4,label='DUAL gauge')

ax.legend(loc="upper right")

ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.legend(loc="upper right")
ax.set_title("Pressure evolution during injection. Corrected for H2")

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm_Sliced df Pressures.png',dpi=300)
plt.show()


In [354]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = func(x,a,b)
ax.plot(inj["Time"],inj["injection 100mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.legend(loc="upper right")
ax.set_xlabel('Timestamp')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.title("Injection pressure evolution 100mbar conductance gauge")
plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm_Sliced df Pressures.png',dpi=300)
plt.show()

In [355]:
#plot coverage vs time
fig, ax = plt.subplots()

ax.plot(inj["Time"],inj["Coverage"],marker=".", markersize=5,label='Coverage')
ax.legend(loc="upper right")
ax.set_xlabel('Timestamp')
ax.set_ylabel('Coverage (M/s*cm2)')
ax.set_yscale('linear')

# Format the x-axis as timestamps
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))

# Rotate and align the x-axis tick labels
fig.autofmt_xdate(rotation=40, ha='right')

plt.title("Coverage over time")
plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S4 a-C H2 ads isotherm_coverage vs time.png',dpi=300)
plt.show()

In [356]:
#plotting pressure vs coverage
fig, ax = plt.subplots()

ax.plot((inj_new["Coverage"]),inj_new["H2 4K BA2"].sub(avg2),marker=".", markersize=5,label='isotherm')
ax.legend(loc="upper left")
#xrange = np.arange(min(inj_new["Coverage"]),max(inj_new["Coverage"]),5e+15)
#ax.set_xticks(xrange)
ax.set_xlim(left = 1e16,right = 3.119e16)

ax.set_xlabel('Coverage (M/cm2)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('log')

plt.title("H2 adsorption isotherm")
plt.tight_layout()  # Ensure all elements are properly placed within the figure       
plt.savefig(r'./graphs/S4 a-C H2 adsorption isotherm.png',dpi=300)
plt.show()

C:\Users\etiirine\AppData\Local\Temp\ipykernel_1940\1412211673.py:12: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  ax.set_yscale('log')


## Writing edited dataframes to file


In [357]:
#desorption yield params
#vaclog
vaclog.to_csv(r'./calculated datasets/vaclogger_output.csv', sep='\t', encoding='utf-8', header=True)

#CERNoX
temps_dict = {"Timestamp":vaclog["Time"],"Temperatures":polyfit(A,vaclog["T-CERNOX"])}
temps = pd.DataFrame(temps_dict)
temps.to_csv(r'./calculated datasets/CERNoX temps.csv', sep='\t', encoding='utf-8', header=True)
#coverage
inj.to_csv(r'./calculated datasets/vaclog during injection.csv', sep='\t', encoding='utf-8', header=True)

#coverage > 1e13
inj_new.to_csv(r'./calculated datasets/cut vaclog during injection.csv', sep='\t', encoding='utf-8', header=True)
